In [1]:
import re

import numpy as np
import pandas as pd

from pathlib import Path
from datetime import datetime

In [2]:
# Carregar o dataframe df_acordaos
start_time = datetime.now()
df_acordaos = pd.read_csv('dados/acordaos.csv', sep=',', error_bad_lines=False, warn_bad_lines=False, low_memory=False)
df_acordaos.rename(str.upper, axis='columns', inplace=True)
print('Duration: {}'.format(datetime.now() - start_time))
df_acordaos.info()

Duration: 0:00:39.217373
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309806 entries, 0 to 309805
Data columns (total 28 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   ACORDAO                           288120 non-null  object 
 1   ADVOGADO                          223477 non-null  object 
 2   ANOACORDAO                        309806 non-null  int64  
 3   ASSUNTO                           77524 non-null   object 
 4   COLEGIADO                         309806 non-null  object 
 5   DATASESSAO                        308360 non-null  object 
 6   DECISAO                           19451 non-null   object 
 7   DECLARACAOVOTO                    3328 non-null    object 
 8   ENTIDADE                          253844 non-null  object 
 9   INTERESSADOS                      255674 non-null  object 
 10  MINISTROALEGOUIMPEDIMENTOSESSAO   5185 non-null    object 
 11  MINISTROAPRESENTOUDECLARACA

In [3]:
df_acordaos.drop(['REPRESENTANTEMP', 'UNIDADETECNICA', 'UNIDADETECNICA', 'MINISTROREVISOR',
                  'MINISTROAUTORVOTOVENCEDOR', 'MINISTROAPRESENTOUDECLARACAOVOTO',
                 'MINISTROALEGOUIMPEDIMENTOSESSAO', 'INTERESSADOS', 'RECURSOS', 'ENTIDADE',
                 'DECLARACAOVOTO', 'ADVOGADO', 'RELATORDELIBERACAORECORRIDA'], axis=1, inplace=True)

df_acordaos.rename(columns={'ANOACORDAO':'ANO_ACORDAO', 'NUMACORDAO':'NUMERO_ACORDAO',
                           'TIPOPROCESSO':'TIPO_PROCESSO', 'DATASESSAO':'DATA_SESSAO',
                           'NUMATA':'NUMERO_ATA', 'PROC':'PROCESSO'}, inplace=True)

df_acordaos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309806 entries, 0 to 309805
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ACORDAO              288120 non-null  object
 1   ANO_ACORDAO          309806 non-null  int64 
 2   ASSUNTO              77524 non-null   object
 3   COLEGIADO            309806 non-null  object
 4   DATA_SESSAO          308360 non-null  object
 5   DECISAO              19451 non-null   object
 6   NUMERO_ACORDAO       309806 non-null  object
 7   NUMERO_ATA           307239 non-null  object
 8   PROCESSO             307121 non-null  object
 9   RELATOR              307262 non-null  object
 10  RELATORIO            110737 non-null  object
 11  SUMARIO              115466 non-null  object
 12  TIPO_PROCESSO        298928 non-null  object
 13  VOTO                 110475 non-null  object
 14  VOTOCOMPLEMENTAR     2517 non-null    object
 15  VOTOMINISTROREVISOR  2925 non-null

In [4]:
df_acordaos.insert(0, 'ID', range(1, len(df_acordaos)+1))
# Transfora em string a coluna ID
df_acordaos['ID'] = df_acordaos['ID'].map(str)
df_acordaos['ANO_ACORDAO'] = df_acordaos['ANO_ACORDAO'].map(str)
df_acordaos.insert(13, 'TIPO_DECISAO', np.NaN)

In [5]:
# DE -> DECISAO = 19451 : 647 marcados como SIGILOSO por esse motivo não possui ACORDAO, VOTO, RELATORIO, SUMARIO
df_acordaos.loc[(df_acordaos.DECISAO.notnull() == True) & ((df_acordaos.ACORDAO.isnull()==True) | (df_acordaos.ACORDAO.str.contains('SIGILOSO', na=False))), 'TIPO_DECISAO'] = 'DE'

In [6]:
# AR -> ACORDAO DE RELACAO : 193963
df_acordaos.loc[(df_acordaos.ACORDAO.notnull()) & (df_acordaos.DECISAO.isnull()) & (df_acordaos.RELATORIO.isnull()) & (df_acordaos.VOTO.isnull()), 'TIPO_DECISAO'] = 'AR'

In [7]:
df_acordaos.loc[df_acordaos.TIPO_DECISAO.isna(), 'TIPO_DECISAO'] = 'AC'

In [8]:
df_acordaos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309806 entries, 0 to 309805
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ID                   309806 non-null  object
 1   ACORDAO              288120 non-null  object
 2   ANO_ACORDAO          309806 non-null  object
 3   ASSUNTO              77524 non-null   object
 4   COLEGIADO            309806 non-null  object
 5   DATA_SESSAO          308360 non-null  object
 6   DECISAO              19451 non-null   object
 7   NUMERO_ACORDAO       309806 non-null  object
 8   NUMERO_ATA           307239 non-null  object
 9   PROCESSO             307121 non-null  object
 10  RELATOR              307262 non-null  object
 11  RELATORIO            110737 non-null  object
 12  SUMARIO              115466 non-null  object
 13  TIPO_DECISAO         309806 non-null  object
 14  TIPO_PROCESSO        298928 non-null  object
 15  VOTO                 110475 non-nu

In [9]:
def format_numero_acordao(valor):
    string = re.sub(re.compile('\D'), '', valor)
    if len(string)>4:
        return string[:-4]
    else:
        return string
    
    
def extrairTipoSessao(texto):
    if len(re.findall('PRIMEIRA', texto)) != 0:
        return '1C'
    elif len(re.findall('SEGUNDA', texto)) > 0:
        return '2C'
    else:
        return 'PL'
    
    
def criaChaveAcordao(row):

    return row.TIPO_DECISAO + '-' + row.NUMERO_ACORDAO.zfill(6) + '-' + str(row.ANO_ACORDAO) + '-' + extrairTipoSessao(row.COLEGIADO.upper())

In [10]:
df_acordaos.NUMERO_ACORDAO = df_acordaos.NUMERO_ACORDAO.apply(format_numero_acordao)

In [11]:
start_time = datetime.now()

df_acordaos.ID = df_acordaos.apply(criaChaveAcordao, axis=1)

print('Duration: {}'.format(datetime.now() - start_time))

Duration: 0:00:07.705488


In [12]:
# 2871 registros com os campos, ACORDAO, DECISAO, VOTO e RELATORIO nulos simultaneamente e por isso excluídos
df_acordaos.drop(df_acordaos.loc[(df_acordaos.ACORDAO.isnull()) & df_acordaos.DECISAO.isnull() & df_acordaos.RELATORIO.isnull() & df_acordaos.VOTO.isnull()].index, inplace=True)

In [13]:
# Registros marcados como SILIGOSO e por esse motivo convertidos para NaN a fim de padronização
df_acordaos.loc[(df_acordaos.ACORDAO.str.lower() == 'sigiloso'), 'ACORDAO'] = np.NaN
df_acordaos.loc[(df_acordaos.DECISAO.str.lower() == 'sigiloso'), 'DECISAO'] = np.NaN
df_acordaos.loc[(df_acordaos.RELATORIO.str.lower() == 'sigiloso'), 'RELATORIO'] = np.NaN
df_acordaos.loc[(df_acordaos.VOTO.str.lower() == 'sigiloso'), 'VOTO'] = np.NaN

In [14]:
df_acordaos.fillna('', inplace=True)

In [15]:
df_acordaos['DECISAO'] = df_acordaos.ACORDAO + ' ' + df_acordaos.DECISAO + ' ' + df_acordaos.RELATORIO + ' ' + df_acordaos.VOTO + ' ' + df_acordaos.VOTOCOMPLEMENTAR + ' ' + df_acordaos.VOTOMINISTROREVISOR

In [16]:
# Retira colunas que não interessam para o trabalho
df_acordaos = df_acordaos.drop(['ANO_ACORDAO', 'ASSUNTO', 'COLEGIADO', 'DATA_SESSAO', 'NUMERO_ACORDAO', 
                                'NUMERO_ATA', 'PROCESSO', 'RELATOR', 'SUMARIO', 'TIPO_PROCESSO', 'ACORDAO', 
                                'VOTO', 'RELATORIO', 'VOTOCOMPLEMENTAR', 'VOTOMINISTROREVISOR', 
                                'TIPO_DECISAO'], axis=1)

df_acordaos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306935 entries, 0 to 309804
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   ID       306935 non-null  object
 1   DECISAO  306935 non-null  object
dtypes: object(2)
memory usage: 7.0+ MB


In [17]:
df_acordaos.sample(2)

,ID,DECISAO
249784,AC-008742-2016-2C,"<p class=""tCU_-_Ac_-_item_9_-_§§"">VISTOS, rela..."
107931,AR-002153-2009-2C,ACÓRDÃO Nº 2153/2009 - TCU - 2ª Câmara</b></p...


In [18]:
df_acordaos.to_csv('dados/df_acordaos.csv', sep='|', index=False)

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)